### 모듈 import 하기

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import re
import urllib.request


import nltk
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
okt = Okt()

from tqdm import tqdm
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [2]:
badcomm = pd.read_csv('./dataset/비속어데이터셋_전처리o.csv')
badcomm

,댓글,비속어여부
0,좌배 까는건,1
1,집에 롱 패딩만 세 개다 10년 더 입어야지,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고,1
...,...,...
6764,후장꽂아,1
6765,후장뚫어,1
6766,흐접,1
6767,흐젚,1


### 토큰화 - 불용어 제거

In [3]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [4]:
# 데이터셋 분리

text = badcomm['댓글'] # 시리즈 객체로 저장
score = badcomm['비속어여부']

### train, test 데이터 분리

In [5]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(text, score , test_size=0.2, random_state=0)
print(len(train_x), len(train_y), len(test_x), len(test_y))

5415 5415 1354 1354


In [6]:
train_x

1833                         딱봐도 남이 인증한거 파온거고 본인거라고도 안했는데
6204                                                  젖꼮찌
4727    리얼로 성정체성에 혼란느껴서 남자인데 남자좋아하는 게이는 전체 1쯤 될듯 근데 인간...
4456                                    개일베잡짓거리하다 니인생종친거다
2876                                    편돌이주제에   누굴 평가해 거
                              ...                        
4931                                                  부랄탁
3264                                               섹스자지보지
1653                                                   팝핀
2607                            민주당 드루킹댓글조작알바 새끼들 정신 못차렸네
2732                디지털이 발전하면서 예견됐던 일이지 근데 여자가 먼저 찍자고 하던데
Name: 댓글, Length: 5415, dtype: object

In [7]:
# 형태소 분석기를 사용하여 토큰화를 하면서 불용어를 제거하여 X_train에 저장

X_train = []
for sentence in tqdm(train_x):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화, 동사 기본형으로 변경
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

100%|██████████| 5415/5415 [00:12<00:00, 439.47it/s]


In [8]:
print(X_train[:3])

[['딱', '보다', '남', '인증', '거', '파', '오다', '보다', '안'], ['젖꼮찌'], ['리얼', '로', '성정체성', '혼란', '느끼다', '남자', '인데', '남자', '좋아하다', '게이', '전체', '1', '쯤', '되다', '근데', '인간', '양성애자', '라', '고함']]


In [9]:
# 형태소 분석기를 사용하여 토큰화를 하면서 불용어를 제거하여 X_test에 저장

X_test = []
for sentence in tqdm(test_x):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화, 동사 기본형으로 변경
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|██████████| 1354/1354 [00:02<00:00, 649.66it/s]


In [10]:
print(X_test[:3])

[['즐', '라도', '사기꾼', '출신', '이라', '던데'], ['리더', '말', '그렇다', '박쥐', '없다', '코로나', '그거', '잡기', '힘들다', '꽈', '래야', '되다', '보지'], ['븅신', '보', '빨다', '나', '치다', '먹다', '이기']]


### 정수 인코딩

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)


In [14]:
# 단어사전 저장!!
import json
with open("일베집합_test4.json","w",encoding="utf-8") as f:
    json.dump(tokenizer.word_index ,f,ensure_ascii=False, indent=4)


In [42]:
# 빈도수가 낮은 단어들은 자연어 처리에서 배제
# 등장 빈도수가 3회 미만인 단어들이 이 데이터에서 얼만큼의 비중을 차지하는지 확인

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 생각보다 희귀단어 등장빈도 비율이 높다.

단어 집합(vocabulary)의 크기 : 10097
등장 빈도가 2번 이하인 희귀 단어의 수: 7190
단어 집합에서 희귀 단어의 비율: 71.20927008022184
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 17.47407273675553


In [43]:
# 등장 빈도수가 2이하인 단어들의 수를 제외한 단어의 개수를 단어 집합의 최대 크기로 제한

# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 2908


In [44]:
# 케라스 토크나이저의 인자로 넘겨주고 텍스트 시퀀스를 정수 시퀀스로 변환

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [45]:
# 정수 인코딩이 잘 되었나?
print(X_train[:3])

[[105, 3, 173, 524, 22, 2184, 58, 3, 11], [], [1430, 10, 430, 119, 62, 119, 190, 120, 857, 92, 767, 5, 68, 158, 50, 698]]


In [46]:
print(train_y)

1833    0
6204    1
4727    0
4456    1
2876    1
       ..
4931    1
3264    1
1653    0
2607    1
2732    0
Name: 비속어여부, Length: 5415, dtype: int64


In [47]:
print(test_y)

1914    0
3433    0
1632    1
4906    0
2323    0
       ..
5872    1
5548    0
1624    1
30      0
6201    1
Name: 비속어여부, Length: 1354, dtype: int64


In [48]:
# y를 array 형식으로 저장해주기

train_y = np.array(train_y)
test_y = np.array(test_y)

In [49]:
print(train_y)
print(test_y )

[0 1 0 ... 0 1 0]
[0 0 1 ... 1 0 1]


### 패딩으로 댓글 길이 맞추기

In [50]:
print('리뷰의 최대 길이 :',max(len(review) for review in train_x))
print('리뷰의 평균 길이 :',sum(map(len, train_x))/len(train_x))

# 최대길이가 요상한 댓글이 있음

리뷰의 최대 길이 : 911
리뷰의 평균 길이 : 26.21865189289012


In [51]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [52]:
max_len = 30
below_threshold_len(max_len, X_train)

# 96 % 가 30 이하이기 때문에 최대글자수는 30으로 패딩해주는것이 좋을것같다.

전체 샘플 중 길이가 30 이하인 샘플의 비율: 96.76823638042474


In [53]:
# 모든 샘플의 길이를 30으로 맞춘다 - 이때 정수인코딩한 데이터 사용!
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

### LSTM으로 분류하기
#### 하이퍼파라미터
- 임베딩 벡터 차원 = 100
- 은닉 상태 크기 = 128
- 배치 크기 = 64
- 에포크 = 15

In [61]:
# 두 개의 선택지 중 하나를 예측하는 이진 분류 문제를 수행하는 모델    

from tensorflow.python.keras.layers import Embedding, Dense, LSTM
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint # 조기종료

embedding_dim = 100
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))# 로지스틱 회귀를 사용해야 하기 때문에 sigmoid

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('test-model1.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, train_y, epochs=15, callbacks=[es, mc], batch_size=16, validation_split=0.2)

Epoch 1/15
269/271 [============================>.] - ETA: 0s - loss: 0.6048 - acc: 0.6731
Epoch 00001: val_acc improved from -inf to 0.76731, saving model to test-model1.h5
271/271 [==============================] - 3s 13ms/step - loss: 0.6031 - acc: 0.6745 - val_loss: 0.5095 - val_acc: 0.7673
Epoch 2/15
268/271 [============================>.] - ETA: 0s - loss: 0.4016 - acc: 0.8223
Epoch 00002: val_acc improved from 0.76731 to 0.76824, saving model to test-model1.h5
271/271 [==============================] - 3s 13ms/step - loss: 0.4020 - acc: 0.8216 - val_loss: 0.4864 - val_acc: 0.7682
Epoch 3/15
271/271 [==============================] - ETA: 0s - loss: 0.3397 - acc: 0.8592
Epoch 00003: val_acc improved from 0.76824 to 0.80609, saving model to test-model1.h5
271/271 [==============================] - 3s 10ms/step - loss: 0.3397 - acc: 0.8592 - val_loss: 0.4517 - val_acc: 0.8061
Epoch 4/15
267/271 [============================>.] - ETA: 0s - loss: 0.3030 - acc: 0.8755
Epoch 00004: va

In [62]:
# 모델 정확도 테스트
loaded_model = load_model('./test-model1.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, test_y)[1]))

43/43 [==============================] - 0s 3ms/step - loss: 0.5046 - acc: 0.8035

 테스트 정확도: 0.8035


### 모델 사용해서 댓글 비속어 포함여부 예측해보기

In [86]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.8):
        print("{:.2f}% 확률로 비속어가 포함된 댓글입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 일반 댓글입니다.\n".format((1 - score) * 100))

In [186]:
slang_predict('미친 존나 싸다!')

'비속어 댓글 99.85% 확률'

In [187]:
slang_predict('졸라 이쁘다 ㅋㅋㅋㅋ')

'비속어 댓글 87.05% 확률'

In [188]:
slang_predict('애새끼가 초딩도 아니고')

'비속어 댓글 97.52% 확률'

In [195]:
slang_predict('시바')

'일반 댓글 40.09% 확률'

In [84]:
test = pd.read_csv('./dataset/153cm_키작녀_시니럽의_봄_여름_코디🎨.csv')
test = test.drop(columns=['id','program','time','love','play'])
test

,comment
0,좋을호 맞넹
1,^^
2,ㅎㅎ
3,👏 👏
4,이옷이 좀 더 날씬해 보여요
5,언니한테 어울려요
6,티 이쁘네영
7,가단이 어떻게 되나영?
8,언니 진짜 다잘어울린다
9,바지 허리밴딩 있나요?


In [125]:
def slang_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', str(new_sentence))
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.8):
        return "비속어 댓글 {:.2f}% 확률".format(score * 100)
    else:
        return "일반 댓글 {:.2f}% 확률".format((1 - score) * 100)

In [89]:
test['result'] = test['comment'].map(lambda x: slang_predict(x))

In [90]:
test

,comment,result
0,좋을호 맞넹,일반 댓글 55.48% 확률
1,^^,일반 댓글 26.69% 확률
2,ㅎㅎ,일반 댓글 26.69% 확률
3,👏 👏,일반 댓글 26.69% 확률
4,이옷이 좀 더 날씬해 보여요,일반 댓글 90.35% 확률
5,언니한테 어울려요,일반 댓글 63.55% 확률
6,티 이쁘네영,일반 댓글 96.25% 확률
7,가단이 어떻게 되나영?,일반 댓글 98.93% 확률
8,언니 진짜 다잘어울린다,일반 댓글 85.06% 확률
9,바지 허리밴딩 있나요?,일반 댓글 88.22% 확률


In [91]:
youtube = pd.read_csv('./dataset/2022-05-11.csv')
youtube= youtube.head(500)

In [92]:
youtube['result'] = youtube['comment'].map(lambda x: slang_predict(x))

In [100]:
youtube.sample(15)

,comment,result
413,원래 똥은 거름 농사시,비속어 댓글 84.83% 확률
116,영국도 핵있다,일반 댓글 75.98% 확률
377,한동훈이가 키다,일반 댓글 26.69% 확률
289,토리가 제일 상식적,일반 댓글 91.32% 확률
295,검수완박 통과못시키면 청외대 20명 감옥간다며 죄지었으면 감옥가야지 왜 양향자의원을...,일반 댓글 39.16% 확률
308,깁시다ㅡ일본은 매일 깁니다,일반 댓글 99.18% 확률
232,동후니가 그러케 무서? 왜 그렇게 무서워? 장관올라가기도 전에 오바야? 만주당 그냥...,일반 댓글 97.75% 확률
415,공산당이 시러요,일반 댓글 26.69% 확률
281,드루킹이 그러더라 문재인에게 조국은 로마다 :shelterin::shelterin:...,일반 댓글 37.14% 확률
304,에잇,일반 댓글 26.69% 확률


In [122]:
youtube2 = pd.read_csv('./dataset/womad_swear.csv', index_col=0)
youtube2

,msg,label
0,지금 어디 계세요,0
1,한국 시간에 시계 맞췄어,0
2,햄버거 두 개랑 콜라 주세요,0
3,우리는 밤새 춤추고 노래했다,0
4,무엇보다도 스트레스를 줄이는 것이 중요합니다,0
...,...,...
1995,개독냄새 나노 ㅋㅋㅋ조팔 개오글,1
1996,사실상 페미니즘 영업사원 아니노 ㅋㅋㅋㅋ,1
1997,ㅋㅋㅋㅋㅋㅋㅋ,1
1998,아버지는 아침 일찍 서울로 가셨다,0


In [126]:
youtube2['result'] = youtube2['msg'].map(lambda x: slang_predict(x))

In [147]:
youtube2.sample(20)

,msg,label,result
518,도와드릴까요 어른 두 명하고 아이 한 명이요,0,일반 댓글 99.31% 확률
1238,이거 전에 올라왓던 글이노,1,일반 댓글 79.31% 확률
799,그걸 이제 알다니 등신력 ㄱㅆㅅㅌㅊ노 근데 저런 개돼지들 군무군무 많아서 정신 못차...,1,비속어 댓글 99.63% 확률
1829,나 새벽에 일어나야 해,0,일반 댓글 93.95% 확률
1452,나를 잊지 않겠다고 약속해,0,일반 댓글 98.40% 확률
620,네네네,1,일반 댓글 39.79% 확률
820,칠판에 써 볼 사람,0,일반 댓글 97.27% 확률
1465,유료회원만 볼 수 있는 댓글입니다,1,일반 댓글 99.31% 확률
1894,저 식당은 냉면으로 유명해,0,일반 댓글 68.09% 확률
1013,후팔ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ나도 돌아왔놐ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ느,1,일반 댓글 96.10% 확률


In [148]:
test2 = pd.read_csv('./dataset/ilbe_swear.csv', index_col=0)
test2['result'] = test2['msg'].map(lambda x: slang_predict(x))

In [174]:
test2.sample(15)

,msg,label,result
3874,ㅈ같은 아메리카요가 본점이든 지점이든 ㅈ되길 바란다,1,일반 댓글 92.76% 확률
4776,네다음용접충,1,일반 댓글 91.46% 확률
3449,꼭 좋은날 올거예요힘내세요ㅜㅜ,0,일반 댓글 99.66% 확률
5576,착오로 송부 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ갈데까지 갔구나...,1,비속어 댓글 97.68% 확률
3319,노무노무 쪽팔리겠다ㅋㅋㅋ,1,비속어 댓글 99.63% 확률
1641,팩트체크 어딘가 모자라고 어딘가 열등감이 있고 결함 있는 사람들만 하위 직종을 무시...,0,일반 댓글 98.04% 확률
494,기레기들아,1,비속어 댓글 84.67% 확률
4637,뭐 좋지 우리야,0,일반 댓글 96.75% 확률
5054,최근에 본 실사화 중에 젤 잘 만든 거 같네 ㄷㄷ,0,일반 댓글 98.40% 확률
3917,네겐 다른 선생이 필요없으니 성령님께서 네 선생님이다,0,일반 댓글 99.31% 확률


In [202]:
slang_predict('너 미친놈이지')

'비속어 댓글 97.60% 확률'